In [1]:
import argparse
import time
import math
import csv
import pandas as pd
from util import *
from layer import *
from trainer import DoubleTrainer
from HierNet import HierarchicalNet
from datetime import datetime

In [2]:
def str_to_bool(value):
    if isinstance(value, bool):
        return value
    if value.lower() in {'false', 'f', '0', 'no', 'n'}:
        return False
    elif value.lower() in {'true', 't', '1', 'yes', 'y'}:
        return True
    raise ValueError(f'{value} is not a valid boolean value')

## --gcn_depth 2 --num_var 4 --var_nodes 4 --stat_nodes 96 --var_node_dim 10 --stat_node_dim 10 --seq_in_len 26 --seq_out_len 1 --batch_size 4 --tanhalpha 2

######################################### Illinois settings ###################################
parser = argparse.ArgumentParser()

parser.add_argument('--device',type=str,default='cpu',help='')
parser.add_argument('--dir',type=str,default='',help='data path')
parser.add_argument('--data_name',type=str,default='Illinois',help='dataset name')

parser.add_argument('--hier_true', type=str_to_bool, default=False, help='whether to use flat graph')
parser.add_argument('--DIL_true', type=str_to_bool, default=True, help='whether to use Dynamic Interaction learning')
parser.add_argument('--gcn_true', type=str_to_bool, default=True, help='whether to add graph convolution layer')
parser.add_argument('--gat_true', type=str_to_bool, default=False, help='whether to add graph attention layer')
parser.add_argument('--buildA_true', type=str_to_bool, default=True, help='whether to construct adaptive adjacency matrix')

parser.add_argument('--gcn_depth',type=int,default=2,help='graph convolution depth')
parser.add_argument('--num_var', type=int, default=4,help='var num during encoding phrase for ablation study') 
parser.add_argument('--var_nodes',type=int,default=4,help='number of nodes/indices')
parser.add_argument('--stat_nodes',type=int,default=96,help='number of nodes/locations')
parser.add_argument('--num_heads',type=int,default=4,help='number of multi-head for graph attention') 
parser.add_argument('--dropout',type=float,default=0.3,help='dropout rate')
parser.add_argument('--var_node_dim',type=int,default=20,help='dim of nodes') 
parser.add_argument('--stat_node_dim',type=int,default=20,help='dim of nodes') 
parser.add_argument('--dilation_exponential',type=int,default=1,help='dilation exponential')

parser.add_argument('--conv_channels',type=int,default=16,help='convolution channels') 
parser.add_argument('--residual_channels',type=int,default=16,help='residual channels') 
parser.add_argument('--skip_channels',type=int,default=32,help='skip channels') 
parser.add_argument('--end_channels',type=int,default=64,help='end channels') 

parser.add_argument('--in_dim',type=int,default=2,help='inputs dimension')  
parser.add_argument('--seq_in_len',type=int,default=26,help='input sequence length')
parser.add_argument('--seq_out_len',type=int,default=1,help='output sequence length') ### can be 1 or not

parser.add_argument('--layers',type=int,default=3,help='number of layers')
parser.add_argument('--batch_size',type=int,default=4,help='batch size')
parser.add_argument('--learning_rate',type=float,default=0.001,help='learning rate')
parser.add_argument('--propalpha',type=float,default=0.05,help='prop alpha')
parser.add_argument('--tanhalpha',type=float,default=1,help='adj alpha')
parser.add_argument('--weight_decay',type=float,default=0.0001,help='weight decay rate')
parser.add_argument('--clip',type=int,default=5,help='clip')

parser.add_argument('--step_size1',type=int,default=1,help='step_size')
parser.add_argument('--step_size2',type=int,default=100,help='step_size')

parser.add_argument('--epochs',type=int,default=20,help='num of train epoch')
parser.add_argument('--print_every',type=int,default=10,help='print information every n iteration')
parser.add_argument('--seed',type=int,default=101,help='random seed')
#parser.add_argument('--patient',type=int,default=10,help='early stop')

parser.add_argument('--expid',type=int,default=1,help='experiment id')
parser.add_argument('--runs',type=int,default=1,help='number of runs')

args, unknown = parser.parse_known_args()
torch.set_num_threads(3)

In [3]:
runid = 1
torch.manual_seed(args.seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(args.seed)

# load data
device = torch.device(args.device)
dir = args.dir

from helper import load_pkl  
from util import DataLoaderM 

def load_Illinois_dataset(dataset_dir, dictname, batch_size, device='cpu'):

    ori_data = load_pkl(dataset_dir, dictname)
    
    data = {}
    for category in ['train', 'val', 'test']:
        tmpilli = ori_data['x_' + category]
        num_year, num_sample, num_variables, num_column = tmpilli.shape  ## num_sample is the number of rows in one year (26*number of counties)

        ## reshape x_category
        allinlist =[]
        num_week = 26
        num_county = 96
        for eachyear in tmpilli:
            for eachweek in eachyear:
                for eachindex in eachweek:
                    allinlist.append(eachindex[3])       
        Yearlist = []
        for i in range(1, num_year +1):
            x = allinlist[(i-1)*num_sample*num_variables : i*num_sample*num_variables]
            Yearlist.append(x)
        Countylist = []
        for eachyear in Yearlist:    
            for i in range(1, num_county +1):
                x = eachyear[(i-1)*num_week*num_variables : i*num_week*num_variables]
                Countylist.append(x)
        Weeklist = []
        for ii in range(20, 45+1):
            for eachcounty in Countylist:
                x = eachcounty[(ii-20)*num_variables : (ii-20+1)*num_variables]
                Weeklist.append(x)
        Weeksequence = []
        for ii in range(1, num_year +1):
            for j in range(1, num_week +1):
                x = Weeklist[(ii-1)*num_county + (j-1)*num_year*num_county : (ii-1)*num_county + (j-1)*num_year*num_county + num_county]
                Weeksequence.append(x)
        Yearweeksequence = []
        for i in range(1, num_year + 1):
            Yearweeksequence.append(Weeksequence[(i-1)*num_week : i*num_week])
        x_array = np.array(Yearweeksequence)
        tmp = x_array
        num_years, num_weeks, num_counties, num_indices = tmp.shape
        tmp = np.expand_dims(tmp, axis=-1)
        tmp_list = [tmp]
        time_ind = [(i % 26) / 26 for i in range(num_weeks)]  # week in year
        time_ind = np.array(time_ind) # 26
        week_in_year = np.tile(time_ind, [num_years, 1, num_counties, num_indices, 1]).transpose((0, 4, 2, 3, 1))
        tmp_list.append(week_in_year)
        data['x_' + category] = np.concatenate(tmp_list, axis=-1)

        ## reshape y_category
        tmp_y = ori_data['y_' + category]
        allyieldlist = []
        for eachline in tmp_y:    
            allyieldlist.append(eachline[2])
        Yearyieldlist = []
        for ii in range(1, num_years +1):
            x = allyieldlist[(ii-1)*num_counties : ii*num_counties]
            Yearyieldlist.append(x)
        y_array = np.array(Yearyieldlist)
        data['y_' + category] = np.expand_dims(y_array, axis=-1) # [year, county, yield]

    data['train_loader'] = DataLoaderM(data['x_train'], data['y_train'], batch_size)
    data['val_loader'] = DataLoaderM(data['x_val'], data['y_val'], batch_size)
    data['test_loader'] = DataLoaderM(data['x_test'], data['y_test'], batch_size)

    return data
    
# Illinois
dataloader = load_Illinois_dataset(dataset_dir = args.dir, dictname = 'illi_data_sepanormal_temporal.dict', batch_size = args.batch_size, device=args.device)

if args.data_name == "Illinois":
   conv_k_size = (1, 4, 1)


model = HierarchicalNet(seq_length=args.seq_in_len, n_var=args.var_nodes, n_stat=args.stat_nodes, var_dim=args.var_node_dim,
                        stat_dim=args.stat_node_dim, device=args.device, tanhalpha=args.tanhalpha, conv_channels=args.conv_channels,
                        gcn_depth=args.gcn_depth, residual_channels=args.residual_channels, in_dim=args.in_dim,
                        dropout=args.dropout, end_channels=args.end_channels, out_dim=args.seq_out_len,
                        propalpha=args.propalpha, predefined_A=args.buildA_true, static_feat=None, dilation_exponential=args.dilation_exponential,
                        layers=args.layers, layer_norm_affline=True, skip_channels=args.skip_channels,
                        gcn_true=args.gcn_true, gat_true=args.gat_true, num_heads = args.num_heads, hier_true=args.hier_true, DIL_true=args.DIL_true, conv_k_size=conv_k_size)

print(args)
print('The receptive field size is', model.receptive_field)
nParams = sum([p.nelement() for p in model.parameters()])
print('Number of model parameters is', nParams)

engine = DoubleTrainer(model, args.learning_rate, args.weight_decay, args.clip, args.step_size1, args.seq_out_len, device=args.device)

print("start training...",flush=True)


illi_data_sepanormal_temporal.dict  is loaded from:  
Namespace(device='cpu', dir='', data_name='Illinois', hier_true=False, DIL_true=True, gcn_true=True, gat_true=False, buildA_true=True, gcn_depth=2, num_var=4, var_nodes=4, stat_nodes=96, num_heads=4, dropout=0.3, var_node_dim=20, stat_node_dim=20, dilation_exponential=1, conv_channels=16, residual_channels=16, skip_channels=32, end_channels=64, in_dim=2, seq_in_len=26, seq_out_len=1, layers=3, batch_size=4, learning_rate=0.001, propalpha=0.05, tanhalpha=1, weight_decay=0.0001, clip=5, step_size1=1, step_size2=100, epochs=20, print_every=10, seed=101, expid=1, runs=1)
The receptive field size is 16
Number of model parameters is 1780870
start training...


In [4]:
## validation loss

his_train_loss = []
his_valid_loss =[]
his_valid_rmse = []
his_valid_pred = []
train_time = []
valid_time = []

for i in range(1,args.epochs+1):# epochs   
    
    train_loss = []
    train_mae = []
    train_mape = []
    train_rmse = []
    t1 = time.time()
    
    dataloader['train_loader'].shuffle() # shuffle train_loader data
    
    for iter, (x, y) in enumerate(dataloader['train_loader'].get_iterator()):
        trainx = torch.Tensor(x).to(device)
        trainx = trainx.transpose(1, 4)
        trainy = torch.Tensor(y).to(device)
        metrics = engine.train(trainx, trainy, iter, data_name=args.data_name)
        train_loss.append(metrics[0])
        train_mae.append(metrics[1])
        train_mape.append(metrics[2])
        train_rmse.append(metrics[3])
    
        if iter % args.print_every == 0 :
            log = 'Iter: {:03d}, Train Loss: {:.4f}, Train MAE: {:.4f}, Train MAPE: {:.4f}, Train RMSE: {:.4f}'
            print(log.format(iter, train_loss[-1], train_mae[-1], train_mape[-1], train_rmse[-1]),flush=True)
    
        t2 = time.time()
        train_time.append(t2-t1)
    
        # validation data
        valid_mae = []
        valid_mape = []
        valid_rmse = []
        valid_pred = []
    
        s1 = time.time()
        for iter, (x, y) in enumerate(dataloader['val_loader'].get_iterator()):
            validx = torch.Tensor(x).to(device)
            validx = validx.transpose(1, 4)
            validy = torch.Tensor(y).to(device)

            if i !=20 :
               metrics = engine.eval(validx, validy, data_name=args.data_name) # first reduce, then expand y:(batch, variables, time)
               valid_mae.append(metrics[0])
               valid_mape.append(metrics[1])
               valid_rmse.append(metrics[2])
            else:    
               metrics = engine.evalbest(validx, validy, data_name=args.data_name) # first reduce, then expand y:(batch, variables, time)
               valid_mae.append(metrics[0])
               valid_mape.append(metrics[1])
               valid_rmse.append(metrics[2])
               valid_pred.append(metrics[3])                    
            
        s2 = time.time()
        log = 'Epoch: {:03d}, Inference Time: {:.4f} secs'
        print(log.format(i,(s2-s1)))
        valid_time.append(s2-s1)
    
        mtrain_loss = np.mean(train_loss)
        mtrain_mape = np.mean(train_mape)
        mtrain_rmse = np.mean(train_rmse)
        his_train_loss.append(mtrain_loss)
    
        mvalid_loss = np.mean(valid_mae)
        mvalid_mape = np.mean(valid_mape)
        mvalid_rmse = np.mean(valid_rmse)
        his_valid_loss.append(mvalid_loss)
        his_valid_rmse.append(mvalid_rmse)
        his_valid_pred.append(valid_pred)
    
        log = 'Epoch: {:03d}, Train Loss: {:.4f}, Train MAPE: {:.4f}, Train RMSE: {:.4f}, Validation Loss: {:.4f}, ' \
              'Validation MAPE: {:.4f}, Validation RMSE: {:.4f}, Training Time: {:.4f}/epoch'
        print(log.format(i, mtrain_loss, mtrain_mape, mtrain_rmse, mvalid_loss, mvalid_mape, mvalid_rmse, (t2 - t1)), flush=True)


Iter: 000, Train Loss: 0.7925, Train MAE: 0.7925, Train MAPE: 1.3832, Train RMSE: 0.8021
Epoch: 001, Inference Time: 7.1540 secs
Epoch: 001, Train Loss: 0.7925, Train MAPE: 1.3832, Train RMSE: 0.8021, Validation Loss: 0.4089, Validation MAPE: 0.8821, Validation RMSE: 0.4301, Training Time: 41.4137/epoch
Epoch: 001, Inference Time: 15.8888 secs
Epoch: 001, Train Loss: 0.7014, Train MAPE: 1.2187, Train RMSE: 0.7134, Validation Loss: 0.2521, Validation MAPE: 0.5433, Validation RMSE: 0.2785, Training Time: 94.0153/epoch
Epoch: 001, Inference Time: 6.5512 secs
Epoch: 001, Train Loss: 0.6108, Train MAPE: 1.0499, Train RMSE: 0.6237, Validation Loss: 0.1709, Validation MAPE: 0.3690, Validation RMSE: 0.2011, Training Time: 146.3410/epoch
Epoch: 001, Inference Time: 15.5130 secs
Epoch: 001, Train Loss: 0.5294, Train MAPE: 0.9054, Train RMSE: 0.5444, Validation Loss: 0.2059, Validation MAPE: 0.4464, Validation RMSE: 0.2639, Training Time: 194.3113/epoch
Epoch: 001, Inference Time: 4.1856 secs
Epo

In [13]:
validpredlist = []
for eachgroup in his_valid_pred[len(his_valid_pred)-1]:
    for eachteam in eachgroup:
        for i in range(0,96):
            validpredlist.append(eachteam[0][i].item())           
validpred = np.array(validpredlist)
validpred = validpred[0:480]

## load yvalid
yvaliddf = pd.read_excel("yvalid.xlsx")
yori_valid = np.array(yvaliddf['Yori'])
ynorm_valid = np.array(yvaliddf['Ystar'])

aa_valid = yori_valid
bb_valid = (np.max(yori_valid) - np.min(yori_valid))*validpred + np.min(yori_valid)
cc_valid = ynorm_valid

# valid R2
expvar1 = np.sum((cc_valid - validpred)**2)
expvar2 = np.sum((cc_valid - np.mean(cc_valid))**2)
validR2 = 1 - expvar1/expvar2

# valid rmse
validrmse = math.sqrt(np.mean(np.square(aa_valid - bb_valid)))

In [14]:
validR2

0.5734944515145726

In [12]:
validrmse

4.59443023578046

In [15]:
storelist = ['Valid'] + list(validpred)
storelist = np.array(storelist)
np.savetxt('Valid.txt', storelist, fmt="%s", delimiter=',')